In [ ]:
### install requirements
!pip install duckdb -qq
!pip install arch -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 10.8 MB/s eta 0:00:00


In [ ]:
# import requirements
import duckdb

import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta, time

import scipy.stats as si
from scipy.optimize import fmin
from arch import arch_model

import contextlib
import io
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
con = duckdb.connect("/content/drive/MyDrive/MFM project/bankruptcy_data.db")

# get price tables
tables = con.execute("SHOW TABLES").df()
# get financial statements
financial = con.execute("SELECT * FROM financial4").df()
# cross check availablity
available = financial[financial['ticker'].isin(tables['name'])]
test = con.execute("Select * from COST").df()
con.close()

In [ ]:
test.head()

,Date,Close
0,2015-07-27,121.071747
1,2015-07-28,121.323540
2,2015-07-29,122.020226
3,2015-07-30,121.953049
4,2015-07-31,121.953049


In [ ]:
test.Date.min()

Timestamp('2015-07-27 00:00:00')

In [ ]:
test.Date.max()

Timestamp('2025-07-25 00:00:00')

In [ ]:
def d1(df, r, T):
    return np.log(df['A'] / (df['total_debt'] * np.exp(-r * T))) / (df['ann_vol'] * np.sqrt(T)) + 0.5 * (df['ann_vol'] * np.sqrt(T))

def findImpliedVals(A, K, sigmaE_E, sigmaA, E, r, T):
    def ImpliedMerton(c):
        d1 = np.log( c[0] / (K * np.exp(-r * T))) / (c[1] * np.sqrt(T)) + 0.5 * (c[1] * np.sqrt(T))
        d2 = d1 - (c[1] * np.sqrt(T))
        f1 = ( c[0] * si.norm.cdf(d1) ) - (K * np.exp(-r * T) * si.norm.cdf(d2)) - E
        f2 =  (si.norm.cdf(d1) * c[1] * c[0] - sigmaE_E) / E
        val = f1**2 + f2**2
        return (val)

    # Suppress optimization output
    with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
        c = fmin(ImpliedMerton, [A, sigmaA], disp=False)

    return c

def kmv_default_prob(asset, dpt, mu_asset, sigma_asset, T):
    numerator = np.log(asset / dpt) + (mu_asset - 0.5 * sigma_asset**2) * T
    denominator = sigma_asset * np.sqrt(T)
    return si.norm.cdf(- numerator / denominator)

def zpp_default_prob(sim_num, prices, horizon):
    np.random.seed(42)
    price_paths = np.zeros((horizon, sim_num))

    # Suppress GARCH model output and warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):

            ## fit AR3-threshold-Garch(1,1) model
            garch_model = arch_model(prices['price_diff'], mean ='AR', lags=3, vol = 'GARCH', p=1, o=1 , q=1)
            garch_model_fit = garch_model.fit()

            ## simulate paths
            forecasts = garch_model_fit.forecast(horizon = horizon, method = 'simulation', simulations = sim_num)

    simulated_changes = forecasts.simulations.values[-1, :, :].T

    for i in range(sim_num):
        for t in range(horizon):
            if t == 0:
                price_paths[t, i] = prices['Close'].iloc[0]
            else:
                price_paths[t, i] = price_paths[t - 1, i] + simulated_changes[t - 1, i]

    ## calculate default probability
    no_default = 0
    for i in range(sim_num):
        path = price_paths[:, i]
        if np.min(path) < 0:
            no_default += 1

    zpp_prob_default = no_default / sim_num # (Number[price path < 0] / total number of simulation)
    return zpp_prob_default

In [ ]:
# con = duckdb.connect("/content/drive/MyDrive/MFM project/bankruptcy_data.db")
# con.execute("DROP TABLE IF EXISTS non_bankrupted_prob_results")
# con.close()

In [ ]:
num_sim = 50_000
horizon = 252
r = 0.01
T = 1

# dataframe to store each ticker's results"
prob_results = pd.DataFrame()

for idx, ticker in enumerate(available['ticker'].unique()):
    start_time = datetime.now()
    # get base data for ticker
    con = duckdb.connect("/content/drive/MyDrive/MFM project/bankruptcy_data.db")
    prices = con.execute(f"SELECT * FROM {ticker}").df()
    financial = con.execute(f"SELECT * FROM financial4 WHERE ticker = '{ticker}'").df()
    bankrupt = con.execute(f"SELECT * FROM non_bankrupted WHERE non_b_ticker = '{ticker}'").df()
    con.close()

    # quarterly placeholder
    quarterly = pd.DataFrame()

    if (prices['Date'].dt.year.min() <= bankrupt.enddate.dt.year.min() - 1):
        # filter until the year before bankrupt
        prices = prices[prices['Date'].dt.year <= bankrupt.enddate.dt.year.min() - 1].copy()
        # calculate log return - for KMV
        prices['log_return'] = np.log(prices['Close'] / prices['Close'].shift(1)).fillna(0)
        # calculate price diff - for ZPP
        prices['price_diff'] = prices['Close'] - prices['Close'].shift(1)
        prices.fillna(0, inplace = True)


        # process by quarter
        prices['quarter_end'] = prices['Date'] + pd.tseries.offsets.QuarterEnd(0)
        quarter_end_price = prices.groupby('quarter_end', as_index = False).last()

        quarterly_vol = prices.groupby('quarter_end', as_index=False).agg({'log_return': 'std'})
        quarterly_vol.columns = ['quarter_end', 'std_log_return']
        quarterly_mean = prices.groupby('quarter_end', as_index=False).agg({'log_return': 'mean'})
        quarterly_mean.columns = ['quarter_end', 'mean_log_return']

        quarterly = quarterly_vol.merge(quarterly_mean, on='quarter_end', how = 'left')

        # merge with financials
        financial['quarter_end'] = pd.to_datetime(financial['date']) + pd.tseries.offsets.QuarterEnd(0)
        quarterly = quarterly.merge(financial, on='quarter_end', how = 'left').dropna()
        quarterly[['shares', 'short_term_debt', 'long_term_debt', 'cash_equivalent']] = quarterly[['shares', 'short_term_debt', 'long_term_debt', 'cash_equivalent']].astype(float)

        # get prices for calculating market cap
        quarterly = quarterly.merge(quarter_end_price[['quarter_end', 'Close']], on = 'quarter_end', how = 'left')

        # process financial statement values
        quarterly['market_cap'] = quarterly['Close'] * quarterly['shares']
        quarterly['total_debt'] = quarterly['short_term_debt'] + quarterly['long_term_debt']
        quarterly['dpt'] = quarterly['short_term_debt'] + 0.5 * quarterly['long_term_debt']
        quarterly['ann_vol'] = quarterly['std_log_return'] * np.sqrt(horizon)
        quarterly['ann_return'] = quarterly['mean_log_return'] * np.sqrt(horizon)

        # drop unnecessary columns
        quarterly = quarterly[['quarter_end', 'market_cap', 'total_debt', 'dpt', 'ann_vol', 'ann_return', 'cash_equivalent']].copy()

        # shift the values by 1 quarter to calculate for information set at that time point
        quarterly[['market_cap', 'total_debt', 'dpt', 'ann_vol', 'ann_return', 'cash_equivalent']] = quarterly[['market_cap', 'total_debt', 'dpt', 'ann_vol', 'ann_return', 'cash_equivalent']].shift(1)
        # shifted values will become NaN
        quarterly.dropna(inplace = True)

        # Let A = asset (assuming total valuation reflect in market_cap), K = total_debt
        quarterly['A'] = quarterly['market_cap'] + quarterly['total_debt'] - quarterly['cash_equivalent']

        # calculate d1, d2
        quarterly['d1'] = d1(quarterly, r, T)
        quarterly['d2'] = quarterly['d1'] - (quarterly['ann_vol'] * np.sqrt(T))

        # calculate Nd1, Nd2
        quarterly['Nd1'] = si.norm.cdf(quarterly['d1'])
        quarterly['Nd2'] = si.norm.cdf(quarterly['d2'])

        # calculate E and sigmaE * E from Merton
        quarterly['E'] = quarterly['A'] * quarterly['Nd1'] - quarterly['total_debt'] * np.exp(-r * T) * quarterly['Nd2']
        quarterly['sigmaE_E'] = quarterly['Nd1'] * quarterly['ann_vol'] * quarterly['A']

        # calculate implied Asset and Asset volatility
        implied_Asset = []
        implied_Vol = []
        for A, K, sigmaE_E, sigmaV, E in zip(quarterly['A'], quarterly['total_debt'], quarterly['sigmaE_E'], quarterly['ann_vol'], quarterly['E']):
            implied_Asset.append(findImpliedVals(A, K, sigmaE_E, sigmaV, E, r, T)[0])
            implied_Vol.append(findImpliedVals(A, K, sigmaE_E, sigmaV, E, r, T)[1])

        quarterly['implied_Asset'] = implied_Asset
        quarterly['implied_Vol'] = implied_Vol

        # calculate KMV probability of default
        quarterly['kmv_prob_default'] = kmv_default_prob(quarterly['implied_Asset'], quarterly['dpt'], quarterly['ann_return'], quarterly['implied_Vol'], T)


        # calculate ZPP probability of default
        zpp_prob_defaults = []
        for quarter in quarterly.quarter_end:
            quarter_prices = prices[prices['quarter_end'] == quarter].copy()
            if (quarter_prices.shape[0] > 0):
                zpp_prob_defaults.append(zpp_default_prob(num_sim, quarter_prices, horizon))
            else:
                zpp_prob_defaults.append(np.nan)
        quarterly['zpp_prob_default'] = zpp_prob_defaults

        quarterly['ticker'] = ticker

        if (quarterly.shape[0] > 0):
            quarterly = quarterly[['ticker', 'quarter_end', 'kmv_prob_default', 'zpp_prob_default']]
            con = duckdb.connect("/content/drive/MyDrive/MFM project/bankruptcy_data.db")
            con.execute("CREATE TABLE IF NOT EXISTS non_bankrupted_prob_results (ticker TEXT, quarter_end DATE, kmv_prob_default FLOAT, zpp_prob_default FLOAT)")
            con.execute(f"INSERT INTO non_bankrupted_prob_results SELECT * FROM quarterly")
            con.close()

    print(f"[{idx + 1}/{available['ticker'].nunique()}] - Processed: {ticker}, Result: {quarterly.shape}, Status: {'Success' * (quarterly.shape[0] > 1)}, Time: {datetime.now() - start_time}")

[1/88] - Processed: CRSP, Result: (49, 4), Status: Success, Time: 0:12:07.039266


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[2/88] - Processed: CPIX, Result: (17, 4), Status: Success, Time: 0:04:06.659338
[3/88] - Processed: COST, Result: (16, 4), Status: Success, Time: 0:03:47.768103
[4/88] - Processed: COP, Result: (12, 4), Status: Success, Time: 0:02:52.363901
[5/88] - Processed: CNTY, Result: (18, 4), Status: Success, Time: 0:04:37.271969
[6/88] - Processed: CNET, Result: (17, 4), Status: Success, Time: 0:04:44.374442
[7/88] - Processed: CLX, Result: (14, 4), Status: Success, Time: 0:03:33.600278
[8/88] - Processed: CLSD, Result: (16, 4), Status: Success, Time: 0:03:49.058482
[9/88] - Processed: CINF, Result: (17, 4), Status: Success, Time: 0:03:56.202523
[10/88] - Processed: CHK, Result: (0, 0), Status: , Time: 0:00:00.070039
[11/88] - Processed: CHCT, Result: (9, 4), Status: Success, Time: 0:02:04.127466


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[12/88] - Processed: CELH, Result: (19, 4), Status: Success, Time: 0:04:35.201007
[13/88] - Processed: CEG, Result: (0, 0), Status: , Time: 0:00:00.080787
[14/88] - Processed: CDW, Result: (15, 4), Status: Success, Time: 0:03:34.903793
[15/88] - Processed: CDNS, Result: (18, 4), Status: Success, Time: 0:04:13.905618
[16/88] - Processed: CDMO, Result: (0, 0), Status: , Time: 0:00:00.072410
[17/88] - Processed: CCZ, Result: (7, 4), Status: Success, Time: 0:01:50.906508
[18/88] - Processed: BIIB, Result: (15, 4), Status: Success, Time: 0:03:37.483159
[19/88] - Processed: BCSF, Result: (15, 4), Status: Success, Time: 0:03:42.858445
[20/88] - Processed: BC, Result: (18, 4), Status: Success, Time: 0:04:17.791901
[21/88] - Processed: BBBY, Result: (0, 0), Status: , Time: 0:00:00.110678
[22/88] - Processed: Acco, Result: (17, 4), Status: Success, Time: 0:04:06.432979
[23/88] - Processed: AXON, Result: (16, 4), Status: Success, Time: 0:03:44.657673
[24/88] - Processed: AVGO, Result: (34, 4), St

...

[67/88] - Processed: MNST, Result: (16, 4), Status: Success, Time: 0:03:49.193561
[68/88] - Processed: LTC, Result: (19, 4), Status: Success, Time: 0:04:41.304141
[69/88] - Processed: LRCX, Result: (16, 4), Status: Success, Time: 0:03:54.560849
[70/88] - Processed: LITE, Result: (17, 4), Status: Success, Time: 0:04:12.013190
[71/88] - Processed: IRDM, Result: (18, 4), Status: Success, Time: 0:04:26.406223
[72/88] - Processed: IDXX, Result: (53, 4), Status: Success, Time: 0:13:41.333881
[73/88] - Processed: IBM, Result: (11, 4), Status: Success, Time: 0:02:52.299464
[74/88] - Processed: HCP, Result: (0, 0), Status: , Time: 0:00:00.098242
[75/88] - Processed: GPS, Result: (0, 0), Status: , Time: 0:00:00.101704


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[76/88] - Processed: GOOG, Result: (17, 4), Status: Success, Time: 0:04:03.375784
[77/88] - Processed: GMBL, Result: (17, 4), Status: Success, Time: 0:03:58.473241
[78/88] - Processed: GBR, Result: (17, 4), Status: Success, Time: 0:04:00.375441
[79/88] - Processed: FCX, Result: (19, 4), Status: Success, Time: 0:04:34.036640
[80/88] - Processed: FBNC, Result: (18, 4), Status: Success, Time: 0:04:17.602337


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[81/88] - Processed: EXEL, Result: (33, 4), Status: Success, Time: 0:07:39.668844
[82/88] - Processed: EQIX, Result: (17, 4), Status: Success, Time: 0:04:01.005219
[83/88] - Processed: ENPH, Result: (37, 4), Status: Success, Time: 0:08:55.584788
[84/88] - Processed: DXCM, Result: (15, 4), Status: Success, Time: 0:03:40.794663
[85/88] - Processed: DXC, Result: (16, 4), Status: Success, Time: 0:03:49.461538
[86/88] - Processed: DGX, Result: (18, 4), Status: Success, Time: 0:04:16.491252
[87/88] - Processed: DG, Result: (16, 4), Status: Success, Time: 0:03:52.218300
[88/88] - Processed: CVGW, Result: (29, 4), Status: Success, Time: 0:06:48.117062
